# Battle of neighbourhood


Here, I want to convert addresses  their equivalent latitude and longitude values. I use the Foursquare API to explore neighborhoods in Toronto , use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. 

I use the k-means clustering algorithm to complete this task. Finally, I use the Folium library to visualize the neighborhoods in Toronto City and their emerging clusters.

This work is organized as:

#### 1. Download and Explore Dataset
#### 2. Explore Neighborhoods in Toronto
#### 3. Analyze Each Neighborhood
#### 4. Cluster Neighborhoods
#### 5. Examine Clusters


In [36]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install geopy
!{sys.executable} -m pip install wget

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [37]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/Users/manashsarma/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


# 1. Download and Explore Dataset

## This is actually a repeat step of last note book where a constructed a dataframe of 5 (3+2) columns after combining two dataframes from wikipedia 'Toronto postal code' table and related geospatial data table 

In [38]:
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_df.head()
import requests
post_codes_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(post_codes_url,'lxml')

# extract data
codes_list=[]
borough_list=[]
neighborhood_list=[]

i=1
for x in soup.table.find_all('td'):
    if i == 1:
        codes_list.append(x.text)
    if i == 2:
        borough_list.append(x.text)
    if i == 3: 
        neighborhood_list.append(x.text)
    i = i+1
    if i==4:
        i=1
    
df = pd.DataFrame(columns=['Postalcode','Borough','Neighborhood'])
df['Postalcode'] = codes_list
df['Borough'] = borough_list
df['Neighborhood'] = neighborhood_list

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
indexs = df[df['Borough']=="Not assigned"].index
df.drop(indexs, inplace=True)

# Also remove newlines
df['Neighborhood'] = df.Neighborhood.str.replace("\n","")
df = df.reset_index(drop=True)

df_modified = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

df_modified.head()
df_merged = pd.concat([df_modified, geo_df], axis = 1).drop('Postal Code', axis=1)

df_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### create map of Toronto using latitude and longitude values

In [39]:
'''import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto'''

"import folium\nmap_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)\n\n# add markers to map\nfor lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighborhood']):\n    label = '{}, {}'.format(neighborhood, borough)\n    label = folium.Popup(label, parse_html=True)\n    folium.CircleMarker(\n        [lat, lng],\n        radius=5,\n        popup=label,\n        color='blue',\n        fill=True,\n        fill_color='#3186cc',\n        fill_opacity=0.7,\n        parse_html=False).add_to(map_toronto)  \n    \nmap_toronto"

### Now, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [40]:
CLIENT_ID = 'SH1AMXRLSPUIOCPWD3TNLOSU1L2EW02RGV24FVWG5SBFIRDJ' # your Foursquare ID
CLIENT_SECRET = 'QCGEQAGXKLXCZ2L1PWGGUBHYFIU43BYJN0UOHJY1X2O4E1Z0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SH1AMXRLSPUIOCPWD3TNLOSU1L2EW02RGV24FVWG5SBFIRDJ
CLIENT_SECRET:QCGEQAGXKLXCZ2L1PWGGUBHYFIU43BYJN0UOHJY1X2O4E1Z0


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [41]:
df_merged.loc[0, 'Neighborhood']

'Rouge,Malvern'

Get the neighborhood's latitude and longitude values.

In [42]:
neighborhood_latitude = df_merged.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_merged.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_merged.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


### Now, let's get the top 100 venues twithin a radius of 1000 meters OR 1 Kilometers.

Let us define URL etc.

In [51]:
#https://developer.foursquare.com/docs/resources/categories
Medical_Center_category = '4bf58dd8d48988d104941735'
Medical_doctoroffice_category = '4bf58dd8d48988d177941735'
Medical_hospital_category = '4bf58dd8d48988d196941735'
Medical_urgentcare_category ='56aa371be4b08b9a8d573526'
Medical_medicallab_category ='4f4531b14b9074f6e4fb0103'
Medical_emergencyroom_category ='4bf58dd8d48988d194941735'
Medical_hospitalWard_category = '58daa1558bbb0b01f18ec1f7'
Mental_health_category = '52e81612bcbc57f1066b7a39'

In [73]:
import requests
LIMIT = 100
radius = 2000

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        Medical_Center_category,
        radius, 
        LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=SH1AMXRLSPUIOCPWD3TNLOSU1L2EW02RGV24FVWG5SBFIRDJ&client_secret=QCGEQAGXKLXCZ2L1PWGGUBHYFIU43BYJN0UOHJY1X2O4E1Z0&v=20180605&ll=43.806686299999996,-79.19435340000001&categoryId=4bf58dd8d48988d104941735&radius=2000&limit=100'

Send the GET request and examine the resutls

In [74]:
results = requests.get(url).json()['response']['groups'][0]['items']

In [75]:
def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

### Assumption in below code
#### If there is a hospital or medical center, there will be a medical lab
#### If there is a medical center, there will be emergency center

In [201]:
Count_Medical_Center_category = 0
Count_Medical_doctoroffice_category=0
Count_Medical_hospital_category=0
Count_Medical_urgentcare_category=0
Count_Medical_medicallab_category=0
Count_Medical_emergencyroom_category=0

for item in results:
    
    cat_id = get_categories(item['venue']['categories'])[0][1]
    
    if(cat_id == Medical_Center_category):
        Count_Medical_Center_category = Count_Medical_Center_category + 1
        Count_Medical_medicallab_category = Count_Medical_medicallab_category + 1
        Count_Medical_emergencyroom_category = Count_Medical_emergencyroom_category + 1
        
        
    if(cat_id == Medical_hospital_category) or (cat_id == Medical_hospitalWard_category):
        Count_Medical_hospital_category = Count_Medical_hospital_category + 1
        Count_Medical_medicallab_category = Count_Medical_medicallab_category + 1
        Count_Medical_doctoroffice_category = Count_Medical_doctoroffice_category + 1
        
    if(cat_id == Medical_doctoroffice_category):
        Count_Medical_doctoroffice_category = Count_Medical_doctoroffice_category + 1
        
    if(cat_id == Medical_urgentcare_category):
        Count_Medical_urgentcare_category = Count_Medical_urgentcare_category + 1
        
    if(cat_id == Medical_medicallab_category):
        Count_Medical_medicallab_category = Count_Medical_medicallab_category + 1 
        
    if(cat_id == Medical_emergencyroom_category):
        Count_Medical_emergencyroom_category = Count_Medical_emergencyroom_category + 1     
    

In [202]:
# Now count number of health care as per type
print( "Number of medical centers is %d\n" % Count_Medical_Center_category)
print( "Number of doctor offices is %d\n" % Count_Medical_medicallab_category)
print( "Number of hospitals is %d\n" % Count_Medical_hospital_category)
print( "Number of medical labs is %d\n" % Count_Medical_medicallab_category)
print( "Number of emrgency centers is %d\n" % Count_Medical_Center_category)

Number of medical centers is 1

Number of doctor offices is 1

Number of hospitals is 1

Number of medical labs is 1

Number of emrgency centers is 1



And how many venues were returned by Foursquare?


## 2. Explore Neighborhoods in Toronto



In [319]:
def nearbyMedicalCenterStats(names, latitudes, longitudes, radius=2000, limit =100):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        Medical_Center_category,
        radius, 
        limit)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        Count_Medical_Center_category = 0
        Count_Medical_doctoroffice_category=0
        Count_Medical_hospital_category=0
        Count_Medical_urgentcare_category=0
        Count_Medical_medicallab_category=0
        Count_Medical_emergencyroom_category=0

        for item in results:

            cat_id = get_categories(item['venue']['categories'])[0][1]

            if(cat_id == Medical_Center_category):
                Count_Medical_Center_category = Count_Medical_Center_category + 1
                Count_Medical_medicallab_category = Count_Medical_medicallab_category + 1
                Count_Medical_emergencyroom_category = Count_Medical_emergencyroom_category + 1
                

            if(cat_id == Medical_hospital_category) or (cat_id == Medical_hospitalWard_category):
                Count_Medical_hospital_category = Count_Medical_hospital_category + 1
                Count_Medical_medicallab_category = Count_Medical_medicallab_category + 1

            if(cat_id == Medical_doctoroffice_category):
                Count_Medical_doctoroffice_category = Count_Medical_doctoroffice_category + 1

            if(cat_id == Medical_urgentcare_category):
                Count_Medical_urgentcare_category = Count_Medical_urgentcare_category + 1

            if(cat_id == Medical_medicallab_category):
                Count_Medical_medicallab_category = Count_Medical_medicallab_category + 1 

            if(cat_id == Medical_emergencyroom_category):
                Count_Medical_emergencyroom_category = Count_Medical_emergencyroom_category + 1
          
        
        venues_list.append(
            [(name, 
            lat, 
            lng, 
            Count_Medical_Center_category,
            Count_Medical_urgentcare_category,  
            Count_Medical_medicallab_category,
            Count_Medical_emergencyroom_category,
            Count_Medical_hospital_category)])
         
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                   'Neighborhood Latitude', 
                   'Neighborhood Longitude', 
                   'Medical Center',
                   'Urgent care',  
                   'Medical Lab',
                   'Medical Emergency',
                   'Hospital']      
    
    return  nearby_venues    

### code below to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [320]:
toronto_venues = nearbyMedicalCenterStats(names=df_merged['Neighborhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )


In [321]:
print(toronto_venues.shape)
toronto_venues

(103, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Medical Center,Urgent care,Medical Lab,Medical Emergency,Hospital
0,"Rouge,Malvern",43.806686,-79.194353,1,0,1,1,0
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,3,0,4,3,1
3,Woburn,43.770992,-79.216917,6,0,8,7,2
4,Cedarbrae,43.773136,-79.239476,6,0,6,7,0
5,Scarborough Village,43.744734,-79.239476,4,0,6,5,2
6,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0,0,0,0,0
7,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,1,0,1,1,0
8,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0,0,0,0,0
9,"Birch Cliff,Cliffside West",43.692657,-79.264848,2,0,2,2,0


In [322]:
## Cluster Neighborhoods
## Run k-means to cluster the neighborhood into 3 clusters.

# set number of clusters
kclusters = 5

healthSupport_clustering = toronto_venues[['Medical Center', 'Urgent care','Medical Lab','Medical Emergency','Hospital']]

healthSupport_clustering.loc[healthSupport_clustering['Medical Center'],'Medical Center' ] = \
                                              healthSupport_clustering['Medical Center'] * 3

healthSupport_clustering['health_care_score'] = toronto_venues['Medical Center'] + toronto_venues['Urgent care'] + toronto_venues['Medical Lab'] \
                    + toronto_venues['Medical Emergency'] + toronto_venues['Hospital']


#health_care_score.reshape(1, -1)
#health_care_score[0:10]
ScoreDF = healthSupport_clustering[['health_care_score']]
ScoreDF.head()


/Users/manashsarma/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/manashsarma/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/manashsarma/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

,health_care_score
0,3
1,0
2,11
3,23
4,19


In [323]:
# run k-means clustering
kmeans = KMeans(n_clusters=3, init='k-means++', n_init=10, max_iter=300, tol=0.0001).fit(ScoreDF)
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0,
       2, 1, 2, 2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [324]:
#Categoried_medical_support=toronto_venues
Categoried_medical_support = toronto_venues
Categoried_medical_support['Medical support category'] = kmeans.labels_

In [325]:
Categoried_medical_support

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Medical Center,Urgent care,Medical Lab,Medical Emergency,Hospital,Medical support category
0,"Rouge,Malvern",43.806686,-79.194353,1,0,1,1,0,0
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,3,0,4,3,1,0
3,Woburn,43.770992,-79.216917,6,0,8,7,2,0
4,Cedarbrae,43.773136,-79.239476,6,0,6,7,0,0
5,Scarborough Village,43.744734,-79.239476,4,0,6,5,2,0
6,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0,0,0,0,0,0
7,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,1,0,1,1,0,0
8,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0,0,0,0,0,0
9,"Birch Cliff,Cliffside West",43.692657,-79.264848,2,0,2,2,0,0


#### Let's check how many venues were returned for each neighborhood

#### Let's find out how many unique categories can be curated from all the returned venues


## 3. Analyze Each Neighborhood

## 4. Cluster Neighborhoods

## 5. Examine Clusters

#### Cluster 1

In [326]:
#Categoried_medical_support.loc[Categoried_medical_support['Medical support category'] == 0, Categoried_medical_support.columns[[0] + list(range(3, Categoried_medical_support.shape[1]))]]
index=Categoried_medical_support['Medical support category'] == 0
Categoried_medical_support[index]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Medical Center,Urgent care,Medical Lab,Medical Emergency,Hospital,Medical support category
0,"Rouge,Malvern",43.806686,-79.194353,1,0,1,1,0,0
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,3,0,4,3,1,0
3,Woburn,43.770992,-79.216917,6,0,8,7,2,0
4,Cedarbrae,43.773136,-79.239476,6,0,6,7,0,0
5,Scarborough Village,43.744734,-79.239476,4,0,6,5,2,0
6,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0,0,0,0,0,0
7,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,1,0,1,1,0,0
8,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0,0,0,0,0,0
9,"Birch Cliff,Cliffside West",43.692657,-79.264848,2,0,2,2,0,0


#### Cluster 2

In [327]:
index=Categoried_medical_support['Medical support category'] == 1
Categoried_medical_support[index]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Medical Center,Urgent care,Medical Lab,Medical Emergency,Hospital,Medical support category
45,Davisville North,43.712751,-79.390197,22,0,34,22,11,1
51,"Cabbagetown,St. James Town",43.667967,-79.367675,25,0,35,25,10,1
52,Church and Wellesley,43.665860,-79.383160,34,0,49,35,15,1
53,Harbourfront,43.654260,-79.360636,26,0,34,26,8,1
54,"Ryerson,Garden District",43.657162,-79.378937,34,0,49,35,15,1
55,St. James Town,43.651494,-79.375418,30,0,45,31,15,1
56,Berczy Park,43.644771,-79.373306,23,0,33,24,10,1
57,Central Bay Street,43.657952,-79.387383,38,0,53,39,15,1
58,"Adelaide,King,Richmond",43.650571,-79.384568,33,0,48,34,15,1
59,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,23,0,31,24,8,1


#### Cluster 3

In [318]:
index=Categoried_medical_support['Medical support category'] == 2
Categoried_medical_support[index]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Medical Center,Urgent care,Medical Lab,Medical Emergency,Hospital,Medical support category
38,Leaside,43.709060,-79.363452,14,0,28,14,28,2
40,East Toronto,43.685347,-79.338106,12,0,16,12,16,2
44,Lawrence Park,43.728020,-79.388790,15,0,29,15,29,2
46,North Toronto West,43.715383,-79.405678,16,0,17,16,16,2
47,Davisville,43.704324,-79.388790,19,0,20,19,19,2
49,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,12,0,12,12,12,2
50,Rosedale,43.679563,-79.377529,12,0,16,12,15,2
63,Roselawn,43.711695,-79.416936,17,0,18,17,17,2
64,"Forest Hill North,Forest Hill West",43.696948,-79.411307,20,0,21,20,20,2
65,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,15,0,22,15,21,2
